In [1]:
import lasair
import numpy as np
import pandas as pd
import json
import yaml
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_columns = 500

In [20]:
def lightcurveSatify(criteria,lightcurve):
    '''
    Our paper states that the tides slection criteria is as follows:
    - only consider griz
    - must have at least 3 5sigma detections
    - Must have 5sigma detections across 2 nights
    - Must reach brighter than 22.5mag
    '''
    needFilters = criteria['filters']
    needSignificance = criteria['significance']
    minBands = criteria['minBands']
    minNights = criteria['minNights']
    magLimit = criteria['magLimit']
    
    filtersBool = np.array([filterDict[x] in inputCriteriaName['filters'] for x in lightcurve['fid']])
    significanceBool = 1.09/lightcurve['sigmapsf'] >= needSignificance
    
    sigAndFilterBool = filtersBool & significanceBool
    
    meetMinBands = len(np.unique(lightcurve['fid'][sigAndFilterBool])) >= minBands
    
    meetMinNight = len(np.unique(lightcurve['nid'][sigAndFilterBool])) >= minNights
    
    meetMagLimit = min(lightcurve['magpsf']) < magLimit
    
    if meetMinBands == meetMinNight == meetMagLimit == True:
        return True
    else:
        return False

def plotLightCurve(name, lc, triggerDate=None, saveName=None):
    if 'diffmaglim' in lc.columns:
        plt.scatter(lc['jd'][nonDets & gIDX], lc['diffmaglim'][nonDets & gIDX], marker='v', color='green', alpha=0.5)
        plt.scatter(lc['jd'][nonDets & rIDX], lc['diffmaglim'][nonDets & rIDX], marker='v', color='red', alpha=0.5)

    plt.errorbar(lc['jd'][gIDX], lc['magpsf'][gIDX], yerr=lc['sigmapsf'][gIDX], fmt='o', color='green', label='g')
    plt.errorbar(lc['jd'][rIDX], lc['magpsf'][rIDX], yerr=lc['sigmapsf'][rIDX], fmt='o', color='red', label='r')
    
    if triggerDate is not None:
        plt.axvline(triggerDate, ls='--', c='k', label='Trigger Date')
    plt.gca().invert_yaxis()
    plt.title(name)
    plt.legend(loc='upper right')
    plt.xlabel('JD')
    plt.ylabel('Mag')
    
    if saveName is not None:
        plt.savefig(saveName, dpi=300, bbox_inches='tight')

In [17]:
##Open Input YAML File
inputCriteriaPath = './tidesSelectionFunctions.yml'
inputCriteriaOpen = yaml.load(open(inputCriteriaPath), Loader=yaml.SafeLoader)
inputCriteriaName = inputCriteriaOpen['tidesSNZTFSelect']

In [8]:
filterDict = {1:'g', 2:'r', 'g':1, 'r':2}

In [2]:
f=open('./42SN-likecandidatesinlast14days.json')
testSne = json.load(f)
numZTFtest = len(testSne['digest'])
ztfNames = np.unique(np.array(([testSne['digest'][i]['objectId'] for i in range(100)]))) #Let's just take the first 100 to not spam the Lasair API

In [3]:
token = 'bb6772e74473f2c7bceb698ae1c377d83b8f7223'
L = lasair.lasair_client(token)
c = L.lightcurves(ztfNames)
#print(c)

In [10]:
passFail = []

In [21]:
for i in range(len(c)): #Change the range to len(c)
    ztfN = ztfNames[i]
    print(ztfN)
    lc = pd.json_normalize(c[i])
    gIDX = lc['fid']==1
    rIDX = lc['fid']==2
    nonDets = lc['candid'].isna()
    FiveSig = 1.09/lc['sigmapsf'] >= 5
    
    ##Does the whole object Pass/Fail our cuts
    wholePF = lightcurveSatify(inputCriteriaName, lc)
    passFail.append(wholePF)
    if wholePF == True:
        ## Now we step through all the detections to test _when_ the object passed
        datesTest = np.unique(lc['jd'][~nonDets])
        pf = np.array(list((map(lambda x: lightcurveSatify(inputCriteriaName, lc[lc['jd']<=x]), datesTest))))
        dateItPasses = min(datesTest[pf])
    else: 
        dateItPasses = None
    
    plotLightCurve(str(ztfN),lc,triggerDate=dateItPasses, saveName='./testOutputFigures/'+str(ztfN)+'.png')
    plt.close()
#listofObjectsPF = np.column_stack((ztfNames,np.array(passFail))

ZTF18aabejsk
ZTF18aagtdcu
ZTF18aampabj
ZTF18aaoinug
ZTF18aatonsq
ZTF18aaudcpk
ZTF18aavvcie
ZTF18aawhnyz
ZTF18aawlxfy
ZTF18aaxsgaq
ZTF18aaybvhx
ZTF18aayltmt
ZTF18abaqetc
ZTF18abcfcka
ZTF18abeainm
ZTF18abeiqpq
ZTF18abeocxc
ZTF18abewqhi
ZTF18abfmpxn
ZTF18abijlar
ZTF18abikohu
ZTF18abjgzxw
ZTF18abjrdjs
ZTF18abkobtt


/home/cf5g09/anaconda3/envs/tides/lib/python3.9/site-packages/numpy/core/_asarray.py:102: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)


ZTF18abkqldw


/home/cf5g09/anaconda3/envs/tides/lib/python3.9/site-packages/numpy/core/_asarray.py:102: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)


ZTF18abkwjcz
ZTF18ablmiep


...

ZTF18ablmjxl


/home/cf5g09/anaconda3/envs/tides/lib/python3.9/site-packages/numpy/core/_asarray.py:102: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)


ZTF18ablrlbm
ZTF18abmhxgi
ZTF18abqavrw
ZTF18abrwpvo
ZTF18abrzevn
ZTF18abrzuwi
ZTF18abseldk
ZTF18absewls
ZTF18absnhwp
ZTF18abucphk
ZTF18abvivzm
ZTF18abvojky
ZTF18abvvkpp
ZTF18abvvkut
ZTF18acauhtv
ZTF18acaxxgr
ZTF18accbkcd
ZTF18accxevi
ZTF18acireod
ZTF18acnzkwo
ZTF19aafoboj
ZTF19aaodvfx


/home/cf5g09/anaconda3/envs/tides/lib/python3.9/site-packages/numpy/core/_asarray.py:102: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)


ZTF19aaphtof
ZTF19aaqapij
ZTF19aardooq
ZTF19aascfdf
ZTF19aaseizn
ZTF19aasgcvz
ZTF19aasgmdx
ZTF19aatvlir
ZTF19aatyrcy
ZTF19aavtoiu
ZTF19abfugkj
ZTF19abjtlwo
ZTF19abkjbnq
ZTF19ablhjzt
ZTF19abpuyms
ZTF19abqmptj
ZTF19abzhubs
ZTF19aculisr
ZTF20aarvuap
ZTF20aasivtu
ZTF20aaurdgf
ZTF20aazwgdx
ZTF20abazcit
ZTF20abfacep
ZTF20abzvaks
ZTF20abzvcun
ZTF20achxjut
ZTF20acinxhb
ZTF20acpcjxu
ZTF20acpolji
ZTF21aalohuy
ZTF21aatlnmw
ZTF21aatlpza
ZTF21aazfoay
ZTF21abfvqyz
ZTF21abrsmnp
ZTF21abscqdw
ZTF22aaafmxt
ZTF22aaafpav
ZTF22aaafpbm
ZTF22aaafqos
ZTF22aaafqsp
ZTF22aaajuvn
ZTF22aaaoxpv
ZTF22aaapcau
ZTF22aaaurkt
ZTF22aaaysmx
ZTF22aabgddv
ZTF22aabosmb
ZTF22aabwzbn
